In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import matplotlib.pyplot as plt
import time
# Directorios para entrenamiento, validación y prueba
train_dir = 'new_dataset/train' 
NP_PATH = 'new_dataset/NP_PATH'

actions = []

# creating data training folders

In [ ]:
## creating folders for the landmarks for each frame of each video
try:
    for action in os.listdir(train_dir):
        action_path = os.path.join(train_dir, action)
        if os.path.isdir(action_path):
            print(f"Processing action: {action}")
            actions.append(action)
    for action in actions:
        action_path=os.path.join(NP_PATH, action)
        os.makedirs(action_path, exist_ok=True)
        print(f'creando:{action_path}')
        for video in range(5):
            video_path=os.path.join(action_path, str(video))
            os.makedirs(video_path, exist_ok=True)
            print(f'creando:{video_path}')
except Exception as e:
    print(e)

# Preprocesing data training

In [18]:
mp_holistic = mp.solutions.holistic  # Modelo Holístico

def draw_landmarks(image, results_holistic):
    mp_drawing = mp.solutions.drawing_utils  # Utilidades de dibujo

    # Configuración para líneas más delgadas
    landmark_drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    connection_drawing_spec = mp_drawing.DrawingSpec(thickness=1)

    if results_holistic.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, results_holistic.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=landmark_drawing_spec,
            connection_drawing_spec=connection_drawing_spec)
    
    if results_holistic.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results_holistic.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=landmark_drawing_spec,
            connection_drawing_spec=connection_drawing_spec)
    
    if results_holistic.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results_holistic.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=landmark_drawing_spec,
            connection_drawing_spec=connection_drawing_spec)
    
    return image

def extract_keypoints(results_holistic):
    # Extracción de keypoints de la pose
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results_holistic.pose_landmarks.landmark]).flatten() if results_holistic.pose_landmarks else np.zeros(33*4)
    
    # Extracción de keypoints de la mano izquierda
    lh = np.zeros(21*3)
    if results_holistic.left_hand_landmarks:
        lh = np.array([[res.x, res.y, res.z] for res in results_holistic.left_hand_landmarks.landmark]).flatten()
    
    # Extracción de keypoints de la mano derecha
    rh = np.zeros(21*3)
    if results_holistic.right_hand_landmarks:
        rh = np.array([[res.x, res.y, res.z] for res in results_holistic.right_hand_landmarks.landmark]).flatten()
    
    return np.concatenate([pose, lh, rh])
    
def process_all_videos(root_path, sequence_length):
    """
    Procesa todos los videos en los subdirectorios de la ruta raíz especificada.
    
    Args:
    root_path (str): Ruta al directorio raíz que contiene las carpetas de videos.
    sequence_length (int): Número de frames a procesar por video.
    NP_PATH (str): Ruta donde se guardarán los archivos numpy con los keypoints.
    """
    # Configuración de MediaPipe
    actions = []

    with mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=2,                 # 0 para más rápido, 2 para más preciso
        smooth_landmarks=True,
        min_detection_confidence=0.6,       # Aumentar el umbral para reducir falsos positivos
        min_tracking_confidence=0.6) as holistic:
        
        for action in os.listdir(root_path):
            action_path = os.path.join(root_path, action)
            if os.path.isdir(action_path):
                print(f"Processing action: {action}")
                num_video =0
                for video in os.listdir(action_path):
                    if video.endswith(('.mp4', '.avi', '.mov')):
                        video_path = os.path.join(action_path, video)
                        process_video(video_path, holistic, action, num_video, sequence_length)
                        num_video+=1
        print("Processed actions:", actions)

def process_video(video_path, holistic, action, video, sequence_length):
    """
    Procesa un único video y extrae los landmarks.
    
    Args:
    video_path (str): Ruta al archivo de video a procesar.
    hands (mediapipe.solutions.hands.Hands): Instancia de MediaPipe Hands.
    pose (mediapipe.solutions.pose.Pose): Instancia de MediaPipe Pose.
    action (str): Nombre de la acción (directorio padre del video).
    video (str): Nombre del archivo de video.
    sequence_length (int): Número de frames a procesar por video.
    NP_PATH (str): Ruta donde se guardarán los archivos numpy con los keypoints.
    """
    cap = cv2.VideoCapture(video_path)
    
    # Crear una ventana para mostrar el video
    window_name = f"Processing: {action} - {video}"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, 640, 720)

    for frame_num in range(sequence_length):
        success, image = cap.read()
        if not success:
            break
        
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results_holistic = holistic.process(image_rgb)
        
        # Dibujar los landmarks en la imagen
        image_with_landmarks = draw_landmarks(image, results_holistic)
        
        # Mostrar la imagen procesada
        cv2.imshow(window_name, image_with_landmarks)
        
        if frame_num == 0:
            cv2.waitKey(500)
        else:
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        keypoints = extract_keypoints(results_holistic)
        npy_path = os.path.join(NP_PATH, action, str(video), str(frame_num))
        np.save(npy_path, keypoints)
    
    cap.release()
    cv2.destroyWindow(window_name)
    print(f"Processed {frame_num + 1} frames from {video_path}")
    # Agregar una pausa de 2 segundos entre videos
    time.sleep(2)




In [ ]:
# running the function

process_all_videos(train_dir, 34) 


In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import pandas as pd
import json


In [13]:
actions = list(pd.Series(actions).unique())
label_map = {label:num for num, label in enumerate(actions)}
with open('datos.json', 'w') as f:
    json.dump(label_map, f)

In [ ]:
label_map

In [ ]:
try:
    sequences, labels = [], []
    for action in actions:
        print(action)
        for sequence in np.array(os.listdir(os.path.join(NP_PATH, action))).astype(int):
            print(sequence)
            window = []
            for frame_num in range(34):
                print(frame_num)
                res = np.load(os.path.join(NP_PATH, action, str(sequence), "{}.npy".format(frame_num)))
                window.append(res)
            sequences.append(window)
            labels.append(label_map[action])
except Exception as e:
    print(e)

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [51]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
val_x, y_val=X, y

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.07)

In [ ]:
y_test.shape

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.regularizers import l2 
from tensorflow.keras.optimizers import Adam
from keras import Input

In [ ]:
print(len(actions))

In [31]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Crear una figura con dos subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))

    # Graficar la pérdida
    ax1.plot(history.history['loss'], label='Pérdida de entrenamiento')
    ax1.plot(history.history['val_loss'], label='Pérdida de validación')
    ax1.set_title('Pérdida del modelo')
    ax1.set_ylabel('Pérdida')
    ax1.set_xlabel('Época')
    ax1.legend(loc='upper right')
    ax1.grid(True)

    # Graficar la precisión
    ax2.plot(history.history['categorical_accuracy'], label='Precisión de entrenamiento')
    ax2.plot(history.history['val_categorical_accuracy'], label='Precisión de validación')
    ax2.set_title('Precisión del modelo')
    ax2.set_ylabel('Precisión')
    ax2.set_xlabel('Época')
    ax2.legend(loc='lower right')
    ax2.grid(True)

    # Ajustar el espacio entre subplots
    plt.tight_layout()

    # Mostrar la figura
    plt.show()



# model

In [134]:

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Definir el callback de Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Métrica a monitorear
    patience=30,         # Número de épocas a esperar antes de detener si no hay mejora
    restore_best_weights=True,  # Restaurar los mejores pesos encontrados
    verbose=1            # Mostrar mensaje cuando se detenga el entrenamiento
)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(34, 258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=False, kernel_regularizer=l2(0.03)))
model.add(BatchNormalization())  # Normalización para mejorar la convergencia
model.add(Dropout(0.5))  # Regularización para evitar overfitting
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.03)))
model.add(Dense(38, activation='softmax'))  # 38 clases




# model = Sequential()
# model.add(Input(shape=(34,258)))
# model.add(LSTM(32, return_sequences=True, activation='relu', ))
# model.add(LSTM(64, return_sequences=True, activation='relu'))
# model.add(BatchNormalization())
# model.add(LSTM(128, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
# model.add(Dense(38, activation='softmax'))
# model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])



model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])



In [ ]:
# Ajustar el modelo con Early Stopping
history = model.fit(X_train, y_train, 
          epochs=200,
          validation_split= 0.1,#(X_train, y_train),  # Usar 20% de los datos para validación
          callbacks=[tb_callback, early_stopping]) #reduce_lr  

# Evaluar en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Loss en test: {test_loss}, Accuracy en test: {test_accuracy}')

# Tensorboard plot

In [ ]:

from tensorboard import program
import webbrowser

# Inicia TensorBoard
tb = program.TensorBoard()

tb.configure(argv=[None, '--logdir', 'Logs/train'])
url = tb.launch()
print(f"TensorBoard iniciado en {url}")


In [ ]:
model.summary()

In [ ]:
results = model.predict(X_test)
results

In [ ]:
actions[np.argmax(results[2])]

In [ ]:
actions[np.argmax(y_test[2])]

In [98]:
model.save('lector_model(95acc-95val_acc-95test).keras')

In [133]:
## deleting model from the buffer
del model

In [105]:
import keras
model=keras.models.load_model('lector_model(99acc-97val_acc).keras')

In [106]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [108]:
ytrue = np.argmax(y_test, axis=1).tolist()

yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [110]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
from collections import deque
import time


last_prediction_time = 0
prediction_interval = 1

sequence = []
sentence = []
predictions = []
threshold = 0.5
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
hands = mp_hands.Hands()
pose = mp_pose.Pose()

# def draw_landmarks(image, results_hands, results_pose):
#     mp_drawing = mp.solutions.drawing_utils
#     mp_hands = mp.solutions.hands
#     mp_pose = mp.solutions.pose
#     if results_pose.pose_landmarks:
#         mp_drawing.draw_landmarks(
#             image, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)
#     if results_hands.multi_hand_landmarks:
#         for hand_landmarks in results_hands.multi_hand_landmarks:
#             mp_drawing.draw_landmarks(
#                 image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
#     return image
def draw_landmarks(image, results_hands, results_pose):
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    mp_pose = mp.solutions.pose

    # Configuración para líneas más delgadas
    landmark_drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    connection_drawing_spec = mp_drawing.DrawingSpec(thickness=1)

    if results_pose.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=landmark_drawing_spec,
            connection_drawing_spec=connection_drawing_spec)
    
    if results_hands.multi_hand_landmarks:
        for hand_landmarks in results_hands.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec=landmark_drawing_spec,
                connection_drawing_spec=connection_drawing_spec)
    
    return image

def extract_keypoints(results_hands, results_pose):
    # # pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results_pose.pose_landmarks.landmark]).flatten() if results_pose.pose_landmarks else np.zeros(33*4)
    
    # # if results_hands.multi_hand_landmarks:
    # #     lh = np.array([[res.x, res.y, res.z] for res in results_hands.multi_hand_landmarks[0].landmark]).flatten()
    # #     if len(results_hands.multi_hand_landmarks) > 1:
    # #         rh = np.array([[res.x, res.y, res.z] for res in results_hands.multi_hand_landmarks[1].landmark]).flatten()
    # #     else:
    # #         rh = np.zeros(21*3)
    # # else:
    # #     lh = np.zeros(21*3)
    # #     rh = np.zeros(21*3)
    
    # # return np.concatenate([pose, lh, rh])
    # pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results_pose.pose_landmarks.landmark]).flatten() if results_pose.pose_landmarks else np.zeros(33*4)
    # lh = np.array([[res.x, res.y, res.z] for res in results_hands.left_hand_landmarks.landmark]).flatten() if results_hands.left_hand_landmarks else np.zeros(21*3)
    # rh = np.array([[res.x, res.y, res.z] for res in results_hands.right_hand_landmarks.landmark]).flatten() if results_hands.right_hand_landmarks else np.zeros(21*3)
    # return np.concatenate([pose, lh, rh])
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results_pose.pose_landmarks.landmark]).flatten() if results_pose.pose_landmarks else np.zeros(33*4)
    
    lh = np.zeros(21*3)
    rh = np.zeros(21*3)
    
    if results_hands.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            hand = np.array([[res.x, res.y, res.z] for res in hand_landmarks.landmark]).flatten()
            if idx == 0:
                lh = hand
            elif idx == 1:
                rh = hand
    
    return np.concatenate([pose, lh, rh])


def prob_viz(res, actions, image, color):
    output_image = image.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_image, (0, 60+num*40), (int(prob*100), 90+num*40), color, -1)
        cv2.putText(output_image, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_image

# Define un solo color para todas las acciones (por ejemplo, azul)
action_color = (255, 0, 0)  # BGR format (Blue)




# Captura de video
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(image_rgb)
    results_pose = pose.process(image_rgb)
    
    # Dibujar los landmarks en la imagen
    image_with_landmarks = draw_landmarks(image, results_hands, results_pose)
    
    # Verificar si se detecta una mano
    if results_hands.multi_hand_landmarks:
        # Extraer keypoints
        keypoints = extract_keypoints(results_hands, results_pose)
        
        # Añadir keypoints a la secuencia y mantener solo los últimos 33
        sequence.append(keypoints)
        sequence = sequence[-33:]
        current_time = time.time()
        if len(sequence) == 33 and current_time - last_prediction_time >= prediction_interval:
            # Realizar predicción
            input_data = np.expand_dims(sequence, axis=0)
            res = model.predict(input_data)[0]
            predicted_action = actions[np.argmax(res)]
            print(predicted_action)
            predictions.append(np.argmax(res))
            sequence.clear()  # Limpiar la secuencia cuando no se detecta mano
            
            # Lógica de visualización
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if predicted_action != sentence[-1]:
                            sentence.append(predicted_action)
                    else:
                        sentence.append(predicted_action)
            
            
            # Visualizar probabilidades
            image_with_landmarks = prob_viz(res, actions, image_with_landmarks, action_color)
            last_prediction_time = current_time
        # Mostrar la predicción en la imagen
        cv2.rectangle(image_with_landmarks, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image_with_landmarks, ' '.join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    else:
        # No se detecta mano
        sequence.clear()  # Limpiar la secuencia cuando no se detecta mano
        cv2.putText(image_with_landmarks, 'No hand detected', (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    # Mostrar la imagen procesada
    cv2.imshow('Real-time Prediction', image_with_landmarks)
    
    # Salir si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [58]:
cap.release()
cv2.destroyAllWindows()